In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import statsmodels.api as sm

In [2]:
df=pd.read_csv("new_train")
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,1998,Medium,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,1987,High,Tier 3,Supermarket Type1,994.7052


In [4]:
df.isnull().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

In [5]:
df.drop(["Item_Identifier",'Outlet_Establishment_Year','Item_Type'],inplace=True,axis=1)

In [6]:
df["Item_Outlet_Sales"].describe()

count     8523.000000
mean      2181.288914
std       1706.499616
min         33.290000
25%        834.247400
50%       1794.331000
75%       3101.296400
max      13086.964800
Name: Item_Outlet_Sales, dtype: float64

In [9]:
X=pd.get_dummies(df.drop("Item_Outlet_Sales",axis=1),drop_first=True)
y=df["Item_Outlet_Sales"]

In [10]:
X.columns

Index(['Item_Weight', 'Item_Visibility', 'Item_MRP',
       'Item_Fat_Content_Regular', 'Outlet_Size_Medium', 'Outlet_Size_Small',
       'Outlet_Location_Type_Tier 2', 'Outlet_Location_Type_Tier 3',
       'Outlet_Type_Supermarket Type1', 'Outlet_Type_Supermarket Type2',
       'Outlet_Type_Supermarket Type3'],
      dtype='object')

In [11]:
from sklearn.model_selection import train_test_split
import statsmodels.api as sm 


In [12]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=3)
from sklearn.preprocessing import PowerTransformer,StandardScaler

# Create a PowerTransformer object
pt =PowerTransformer()

# Fit and transform the X_train data
X_train_transformed = pd.DataFrame(pt.fit_transform(X_train),columns=X_train.columns)

# Only transform the X_test data, using the same transformer that was fit to the X_train data
X_test_transformed = pd.DataFrame(pt.transform(X_test),columns=X_test.columns)


In [13]:
from xgboost import XGBRegressor

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

# Create a LinearRegression object
lin_reg = LinearRegression()

# Create an RFE object with the desired number of features
rfe = RFE(lin_reg, n_features_to_select=7)

# Fit the RFE object to the data
rfe.fit(X_train_transformed, y_train)

# Get the selected features
selected_features = X_train_transformed.columns[rfe.support_]
selected_features

Index(['Item_MRP', 'Item_Fat_Content_Regular', 'Outlet_Size_Small',
       'Outlet_Location_Type_Tier 3', 'Outlet_Type_Supermarket Type1',
       'Outlet_Type_Supermarket Type2', 'Outlet_Type_Supermarket Type3'],
      dtype='object')

In [14]:
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from catboost import CatBoostRegressor
import numpy as np

# list of regression algorithms
regressors = [LinearRegression(), Lasso(), Ridge(), DecisionTreeRegressor(), RandomForestRegressor(), 
              XGBRegressor(), SVR(), KNeighborsRegressor()]

# loop through the list and fit the model, then predict and calculate metrics
metrics = []
for reg in regressors:
    reg.fit(X_train[selected_features], y_train)
    y_pred = reg.predict(X_test[selected_features])
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred,squared=False)
    r2 = r2_score(y_test, y_pred)
    metrics.append([type(reg).__name__, mae, mse, r2])

# create dataframe and print metrics
df = pd.DataFrame(metrics, columns = ['Algorithm', 'MAE', 'MSE', 'R2'])
print(df)


               Algorithm          MAE          MSE        R2
0       LinearRegression   815.142604  1096.526221  0.596219
1                  Lasso   814.415211  1096.456553  0.596271
2                  Ridge   814.908303  1096.510171  0.596231
3  DecisionTreeRegressor  1030.437117  1528.142878  0.215785
4  RandomForestRegressor   842.494712  1221.583845  0.498866
5           XGBRegressor   786.870651  1145.607197  0.559264
6                    SVR  1126.983206  1551.556748  0.191569
7    KNeighborsRegressor   918.923258  1276.363339  0.452913


In [15]:
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from catboost import CatBoostRegressor
import numpy as np

# list of regression algorithms
regressors = [LinearRegression(), Lasso(), Ridge(), DecisionTreeRegressor(), RandomForestRegressor(), 
              XGBRegressor(), SVR(), KNeighborsRegressor()]

# loop through the list and fit the model, then predict and calculate metrics
metrics = []
for reg in regressors:
    reg.fit(X_train_transformed[selected_features], y_train)
    y_pred = reg.predict(X_test_transformed[selected_features])
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred,squared=False)
    r2 = r2_score(y_test, y_pred)
    metrics.append([type(reg).__name__, mae, mse, r2])

# create dataframe and print metrics
df = pd.DataFrame(metrics, columns = ['Algorithm', 'MAE', 'MSE', 'R2'])
print(df)


               Algorithm          MAE          MSE        R2
0       LinearRegression   817.076346  1100.524770  0.593269
1                  Lasso   816.855396  1100.526864  0.593268
2                  Ridge   817.049543  1100.527887  0.593267
3  DecisionTreeRegressor  1025.150158  1520.283883  0.223830
4  RandomForestRegressor   844.764605  1224.586743  0.496399
5           XGBRegressor   787.501344  1145.905261  0.559034
6                    SVR  1176.426084  1629.683888  0.108104
7    KNeighborsRegressor   790.245791  1136.938359  0.565909


In [16]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

# Create a Lasso object
lasso = Lasso()

# Define the parameter grid for alpha
param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10]}

# Create a GridSearchCV object
grid_search = GridSearchCV(lasso, param_grid, cv=5, scoring='neg_root_mean_squared_error')

# Fit the GridSearchCV object to the data/
grid_search.fit(X_train, y_train)

# Print the best parameters and the best score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)


Best parameters:  {'alpha': 0.1}
Best score:  -1138.4488295026767
